# Constructive heuristics comparison

The objective of the $\alpha$-neighbor $p$-center problem can be thought of as distributing the facilities among the clients to cover them efficiently, which is the actual goal of the $p$-center problem, so a constructive heuristic that uses its objective function will be tested and compared against a greedy heuristic that takes into account the objective function of this problem.

There will be used 20 random instances of size $n = 50$, $p = 5$ and other 20 of size $n = 400$, $p = 20$, and each one will be tested with both $\alpha = 2$ and $\alpha = 3$. The coordinates of the points are between 0 and 1000 for both planes.

In [2]:
from copy import deepcopy
from typing import List

from models.instance import Instance


def generate_instances(amount: int, n: int, p: int) -> List[Instance]:
    alpha2 = [
        Instance.random(n, p, 2, 1000, 1000)
        for _ in range(amount)
    ]
    alpha3 = deepcopy(alpha2)
    for i in alpha3:
        i.alpha = 3
    return alpha2 + alpha3

In [3]:
instances = generate_instances(20, 50, 5) + generate_instances(20, 400, 20)

We will use the following code to measure the time taken by the evaluations and the objective function results, formatted in a Pandas DataFrame.

In [2]:
import timeit

import pandas as pd

from heuristics.constructive import pdp_based, greedy
from utils import eval_obj_func

def measure(instance, heuristic):
    start = timeit.default_timer()
    solution = heuristic(instance)
    time = timeit.default_timer() - start
    of = eval_obj_func(instance, solution)
    return heuristic.__name__, solution, of, time

def get_dataframe(data):
    return pd.DataFrame({
        colname: [d[i] for d in data]
        for colname, i in zip(
            ('n', 'p', 'a', 'heuristic', 'solution', 'OF', 'seconds'),
            range(len(data[0])))
    })

## Comparing data

In [7]:
import os

OUT_FOLDER = 'nb_results\\constructive'

filepath = os.path.join(OUT_FOLDER, 'pdp_df.csv')
if os.path.exists(filepath):
    pdp_df = pd.read_csv(filepath)
else:
    pdp_data = [(*i.get_parameters(), *measure(i, pdp_based))  for i in instances]
    pdp_df = get_dataframe(pdp_data)
    pdp_df.to_csv(filepath, index=False)
pdp_df

,n,p,a,heuristic,solution,OF,seconds
0,50,5,2,pdp_based,"{33, 5, 37, 40, 28}",628,0.000760
1,50,5,2,pdp_based,"{33, 2, 34, 13, 14}",607,0.000535
2,50,5,2,pdp_based,"{35, 5, 8, 9, 42}",794,0.000527
3,50,5,2,pdp_based,"{18, 19, 13, 45, 14}",611,0.000617
4,50,5,2,pdp_based,"{1, 7, 27, 29, 47}",643,0.000525
...,...,...,...,...,...,...,...
75,400,20,3,pdp_based,"{66, 68, 198, 74, 395, 397, 399, 16, 81, 209, ...",406,0.074006
76,400,20,3,pdp_based,"{261, 8, 201, 202, 13, 22, 281, 155, 222, 289,...",390,0.093924
77,400,20,3,pdp_based,"{1, 259, 325, 198, 134, 201, 204, 77, 78, 208,...",375,0.073400
78,400,20,3,pdp_based,"{391, 10, 13, 77, 271, 16, 17, 274, 144, 84, 2...",372,0.064396


Saving the evaluation to a CSV:

In [17]:
filepath = os.path.join(OUT_FOLDER, 'greedy_df.csv')
if os.path.exists(filepath):
    greedy_df = pd.read_csv(filepath)
else:
    greedy_data = [(*i.get_parameters(), *measure(i, greedy))  for i in instances]
    greedy_df = get_dataframe(greedy_data)
    greedy_df.to_csv(filepath, index=False)
greedy_df

,n,p,a,heuristic,solution,OF,seconds
0,50,5,2,greedy,"{2, 3, 37, 40, 28}",600,0.216955
1,50,5,2,greedy,"{33, 1, 13, 14, 15}",607,0.067130
2,50,5,2,greedy,"{1, 35, 8, 44, 31}",627,0.021097
3,50,5,2,greedy,"{18, 3, 4, 43, 13}",591,0.050969
4,50,5,2,greedy,"{20, 38, 7, 9, 29}",610,0.020568
...,...,...,...,...,...,...,...
75,400,20,3,greedy,"{1, 2, 3, 4, 5, 198, 6, 8, 7, 9, 10, 14, 16, 2...",369,32.796715
76,400,20,3,greedy,"{1, 2, 3, 4, 5, 6, 7, 8, 201, 10, 12, 14, 16, ...",401,36.384875
77,400,20,3,greedy,"{1, 2, 3, 67, 4, 6, 201, 9, 11, 12, 77, 78, 14...",434,33.087493
78,400,20,3,greedy,"{1, 2, 3, 4, 5, 6, 7, 8, 9, 138, 10, 12, 13, 1...",404,29.960488


Calculating statistics of the results:

In [23]:
filtered_data = {
    heuristic: {
        f'n{n}': {
            f'a{alpha}': df[
                (df['n'] == n) &
                (df['a'] == alpha)
                ].iloc[:, [0, 1, 2, 3, 5, 6]]
            for alpha in (2, 3)
        }
        for n in (50, 400)
    }
    for heuristic, df in (('pdp', pdp_df), ('greedy', greedy_df))
}

In [235]:
stats = (filtered_data['pdp']['n400']['a2']
    .compare(filtered_data['greedy']['n400']['a2'], keep_equal=True)
    .rename(columns={ 'self': 'pdp', 'other': 'greedy' })
    .drop(columns='heuristic'))

stats['OF', 'absolute'] = stats['OF', 'pdp'] - stats['OF', 'greedy']
stats['OF', 'relative'] = (stats['OF', 'absolute'] / stats['OF', 'pdp']).map(lambda x: f'{x:.2%}')

stats['seconds', 'absolute'] = stats['seconds', 'pdp'] - stats['seconds', 'greedy']
stats['seconds', 'relative'] = (stats['seconds', 'absolute'] / stats['seconds', 'pdp']).map(lambda x: f'{x:.2%}')

order = ['pdp', 'greedy', 'absolute', 'relative']
# stats.sort_index(axis=1, inplace=True)
stats = stats.loc[:, (('OF', 'seconds'), order)]
# stats['OF'].loc[:, order]

stats

d:\NETO\programacion\alpha-neighbor-p-center-problem\venv\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


OF                            seconds                                   
    pdp greedy absolute relative       pdp     greedy   absolute     relative
40  307    304        3    0.98%  0.071640  45.307980 -45.236340   -63144.06%
41  292    350      -58  -19.86%  0.102789  42.235033 -42.132243   -40988.90%
42  310    309        1    0.32%  0.125210  50.214393 -50.089183   -40004.11%
43  294    426     -132  -44.90%  0.051584  45.909697 -45.858113   -88900.74%
44  317    297       20    6.31%  0.057793  48.009938 -47.952144   -82971.81%
45  297    358      -61  -20.54%  0.049126  51.167755 -51.118629  -104055.95%
46  328    336       -8   -2.44%  0.051110  45.846795 -45.795684   -89601.32%
47  323    354      -31   -9.60%  0.050484  45.359293 -45.308809   -89747.96%
48  275    362      -87  -31.64%  0.052707  49.758392 -49.705685   -94305.48%
49  339    341       -2   -0.59%  0.062176  50.711559 -50.649382   -81460.78%
50  304    277       27    8.88%  0.054250  48.212123 -48.157873   -88769.78%
51  324    362      -38  -11.73%  0.057710  51.083328 -51.025618   -88416.68%
52  294    315      -21   -7.14%  0.053468  50.279512 -50.226045   -93937.52%
53  312    336      -24   -7.69%  0.056241  58.758920 -58.702679  -104376.83%
54  276    308      -32  -11.59%  0.057420  55.484057 -55.426636   -96528.11%
55  348    302       46   13.22%  0.053719  52.859262 -52.805543   -98300.30%
56  359    299       60   16.71%  0.057072  44.194823 -44.137751   -77337.37%
57  343    348       -5   -1.46%  0.056023  50.252007 -50.195984   -89598.25%
58  276    348      -72  -26.09%  0.058349  38.374765 -38.316416   -65667.99%
59  284    270       14    4.93%  0.058375  43.111878 -43.053502   -73752.82%